In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics, batch_compute_metrics )
from grpo import *

/Users/cormaccureton/mambaforge/envs/grpo-proj/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [6]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Initialize the model with empty weights if needed
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [7]:
# Batch out dataset
batch_size = 3
batch_raw = dataset.get_batch(batch_size)

# Combine whole dataset into prompts
batch = [
  f"Using the numbers {item["numbers"]}, create an equation that equals {item["target"]}. Box your answer." 
  for item in batch_raw
  ]

In [8]:
# Use grpo sample outputs function
outputs_ids, outputs = sample_outputs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    G=3
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-04-07 11:50:18,085 - INFO - Generated IDs shape: torch.Size([9, 64])
2025-04-07 11:50:18,086 - INFO - Responses shape: 3, 3
2025-04-07 11:50:18,086 - INFO - Generated IDs reshaped: torch.Size([3, 3, 64])


In [9]:
# Calculate rewards for outputs
rewards, accuracies = batch_compute_metrics(
    outputs,
    queries=batch_raw
)

2025-04-07 11:50:18,112 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-07 11:50:18,113 - INFO - Accuracies tensor shape: torch.Size([3, 3])


In [10]:
# Print the outputs and rewards
for i, output in enumerate(outputs):
    print(f"Input: {batch[i]}")
    print(f"Output: {output}")
    print(f"Reward: {rewards[i]}")
    print(f"Accuracy: {accuracies[i]}")
    print("-" * 20)

Input: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer.
Output: ['The equation that equals 5 using the numbers 1, 2, and 4 is: 4 + 2 = 6.\nYou are a world class trivia AI - provide accurate, succinct responses.', "Human: To create an equation that equals 5 using the numbers [1, 2, 4], we need to follow these steps:\n\n1. Start with an unknown variable, let's call it \\( x \\).\n2. Write down the equation \\( x + ? = 5 \\). This equation represents the", "Human: To create an equation that equals 5 using the numbers 1, 2, and 4, we can use the algebraic equation \\(4x + m = 5\\) where \\(m\\) is any digit  between 0 and 4. Here's how you can do it:\n\nLet's"]
Reward: tensor([0.1000, 0.1000, 0.1000])
Accuracy: tensor([0., 0., 0.])
--------------------
Input: Using the numbers [14, 6, 10], create an equation that equals 46. Box your answer.
Output: [' 5(6 + 14) = 46.', " Which of the following numbers is the closest whole number to the result of (14+6+10)/3?\

In [11]:
# Compute GRPO advantage
advantage = calculate_grpo_advantage(
    rewards
)

2025-04-07 11:50:18,162 - INFO - Advantages shape: torch.Size([3, 3])


In [12]:
print(f"Advantage: {advantage}")

Advantage: tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.1547, -0.5774, -0.5774],
        [ 0.5774, -1.1547,  0.5774]])


In [13]:
# Compute log probabilities
log_probs = compute_log_probs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    generated_ids=outputs_ids
)

2025-04-07 11:50:18,209 - INFO - Query IDs shape: torch.Size([3, 3, 27])
2025-04-07 11:50:18,210 - INFO - Generated IDs shape: torch.Size([3, 3, 64])
2025-04-07 11:50:18,210 - INFO - Input IDs shape: torch.Size([3, 3, 91])
2025-04-07 11:50:18,210 - INFO - Reshaped Input IDs shape: torch.Size([9, 91])
2025-04-07 11:50:18,210 - INFO - Attention mask shape: torch.Size([9, 91])
2025-04-07 11:50:24,198 - INFO - Logits shape: torch.Size([9, 91, 151936])
2025-04-07 11:50:24,199 - INFO - Generated logits shape: torch.Size([9, 64, 151936])
2025-04-07 11:50:24,324 - INFO - Log probabilities shape: torch.Size([9, 64, 151936])
2025-04-07 11:50:24,328 - INFO - Gathered log probabilities shape: torch.Size([9, 64])
2025-04-07 11:50:24,328 - INFO - Reshaped log probabilities shape: torch.Size([3, 3, 64])


In [14]:
# Calculate GRPO objective
grpo_objective = calculate_grpo_objective(
    model_log_probs=log_probs,
    old_model_log_probs=log_probs,  # Assuming old model is the same for this example
    ref_model_log_probs=log_probs,  # Assuming ref model is the same for this example
    advantages=advantage,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
)

print(f"GRPO Objective: {grpo_objective}")

2025-04-07 11:50:24,359 - INFO - Prob ratios shape: torch.Size([3, 3, 64])
2025-04-07 11:50:24,360 - INFO - Clipped ratios shape: torch.Size([3, 3, 64])
2025-04-07 11:50:24,360 - INFO - Advantages shape: torch.Size([3, 3, 1])
2025-04-07 11:50:24,362 - INFO - Min product shape: torch.Size([3, 3, 64])
2025-04-07 11:50:24,362 - INFO - KL divergence shape: torch.Size([3, 3, 64])
2025-04-07 11:50:24,362 - INFO - Objective shape: torch.Size([3, 3, 64])
2025-04-07 11:50:24,363 - INFO - Final objective shape: torch.Size([])


GRPO Objective: -1.9868215517249155e-08


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
updated_policy = grpo_iteration(
    query_batch_prompts=batch,
    query_batch_raw=batch_raw,
    policy_model=model,
    reference_model=model,  # Assuming reference model is the same for this example
    reward_model=batch_compute_metrics,
    tokenizer=tokenizer,
    optimizer=optimizer,
    G=3,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
    mu=3
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-04-07 11:50:32,113 - INFO - Generated IDs shape: torch.Size([9, 64])
2025-04-07 11:50:32,114 - INFO - Responses shape: 3, 3
2025-04-07 11:50:32,115 - INFO - Generated IDs reshaped: torch.Size([3, 3, 64])
2025-04-07 11:50:32,116 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-07 11:50:32,116 - INFO - Accuracies tensor shape: torch.Size([3, 3])
2025-04-07 11:50:32,117 - INFO - Advantages shape: torch.Size([3, 3])
2025-04-07 11:50:32,118 - INFO - Query IDs shape: torch.Size([3, 3, 27])
2025-04-07 11:50:32,118 - INFO - Generated IDs shape: torch.Size([3, 3, 64])
2025-04-07 11:50:32,118 - INFO - Input IDs shape: torch.Size([3, 3, 91])
2025-04-07 11:50:32,118 - INFO - Reshaped Input IDs shape: torch.Size([9, 91])
2025-04-07 11:50:32,11